# Container Apps - File share Deployment
This template sets up a containerized application environment with necessary Azure resources, including a file share linked to the container's volumn mount, Azure Container Registry access, container deployment, storage account, and specified role assignments. Please ensure an Container Environment is created before proceeding with the deployment, see [/scenario-container-app-environment.ipynb](../container-app-environment/scenario-container-app-environment.ipynb).

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- Container app environment ([See scenario-container-app-environment.ipynb](../container-app-environment/scenario-container-app-environment.ipynb))


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [ ]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=northeurope

TEMPLATE_FILE = "main.bicep"
PARAMETERS_FILE = "main.bicepparam"
MANAGED_ENVIRONMENT_NAME=my-container-app-env
MANAGED_ENVIRONMENT_RESOURCE_GROUP=container-apps
REGISTRY_NAME=lenndewoltentestacr
REGISTRY_RESOURCE_GROUP=container-apps

2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "container-apps-deployment"
template_file = os.getenv('TEMPLATE_FILE')
parameters_file = os.getenv('PARAMETERS_FILE')

managed_environment_name = os.getenv('MANAGED_ENVIRONMENT_NAME')
managed_environment_resource_group = os.getenv('MANAGED_ENVIRONMENT_RESOURCE_GROUP')
registry_name = os.getenv('REGISTRY_NAME')
registry_resource_group = os.getenv('REGISTRY_RESOURCE_GROUP')

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

print(f"Managed Environment Name: {managed_environment_name}")
print(f"Managed Environment Resource Group: {managed_environment_resource_group}")
print(f"Registry Name: {registry_name}")
print(f"Registry Resource Group: {registry_resource_group}")

Subscription ID: 7ba9e6b6-7591-4828-aaa5-c4656cb871c3
Resource Group: functionapp-on-container-apps
Location: northeurope
Managed Environment Name: my-container-app-env
Managed Environment Resource Group: container-apps
Registry Name: None
Registry Resource Group: None


4. Login and set default Azure subscription

In [2]:
!az login
!az account set --subscription  {subscription_id}

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "id": "835ebe2e-68d1-4817-925d-1aae42ec01db",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "37ac251a-9341-4ae8-b2a2-aa7febf5ce05"
      }
    ],
    "name": "RBR_DPT_GENERIC_P",
    "state": "Enabled",
    "tenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "user": {
      "name": "ROB5207@robeco.nl",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "id": "e84d14c9-e150-4500-93b2-5aa2829e295d",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "37ac251a-9341-4ae8-b2a2-aa7febf5ce05"
      }
    ],
    "name": "RBR_CSS_EQUITY_O",
    "state": "Enabled",
    "tenantId": "71dd74e2-a620-4a8e-9ac4-a19e1ff9ddff",
    "user": {
      "name": "ROB5207@robeco.nl",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "7

5. Create an Azure Resource group:

In [3]:
!az group create --name {resource_group} --location {location}

{
  "id": "/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/functionapp-on-container-apps",
  "location": "northeurope",
  "managedBy": null,
  "name": "functionapp-on-container-apps",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [4]:
%%writefile {parameters_file}

using '#{template_file}#'

param containerAppEnvName = '#{managed_environment_name}#'
param containerAppEnvRG = '#{managed_environment_resource_group}#'

param resourceNamePrefix = uniqueString('lenny')

Overwriting main.bicepparam


2. Seed container registry with the docker image

In [ ]:
source = 'mcr.microsoft.com/azuredocs/containerapps-helloworld:latest'
target = 'containerapps-helloworld:latest'

!az acr import --name {registry_name} --source {source} --image {target} --force

3. Replace tokens in `.bicepparam` file

In [5]:
%run "../../../utils/placeholder_replacer.py"

path = f'{parameters_file}'
replace_placeholders_in_file(path)

Replacing key: "template_file", with value: "main.bicep"
Replacing key: "managed_environment_name", with value: "my-container-app-env"
Replacing key: "managed_environment_resource_group", with value: "container-apps"


4. Deploy the template:

In [6]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name}


{
  "id": "/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/functionapp-on-container-apps/providers/Microsoft.Resources/deployments/container-apps-deployment",
  "location": null,
  "name": "container-apps-deployment",
  "properties": {
    "correlationId": "1946595a-cda8-4750-bc6f-92c50acd7243",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/functionapp-on-container-apps/providers/Microsoft.OperationalInsights/workspaces/v32kvrsn2pszm-la",
            "resourceGroup": "functionapp-on-container-apps",
            "resourceName": "v32kvrsn2pszm-la",
            "resourceType": "Microsoft.OperationalInsights/workspaces"
          }
        ],
        "id": "/subscriptions/7ba9e6b6-7591-4828-aaa5-c4656cb871c3/resourceGroups/functionapp-on-container-apps/providers/Microsoft.Insights/components/v32kvrsn2pszm-ai",
        "resourceGroup": "functi

5. Open the app!

In [ ]:
import os
import webbrowser

fqdn = os.popen(f"az deployment group show --resource-group {resource_group} --name {deployment_name} --query \"properties.outputs.fqdn.value\" --output tsv").read().strip()

print(f"FQDN: {fqdn}")

webbrowser.open_new(fqdn)